In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
root_dir = 'C:\\Users\\cgrigsby\\OneDrive - University of Tennessee\\Documents\\GitHub\\'
data_dir = root_dir + 'agec-784\\data\\'
solar_data = pd.read_csv(data_dir + 'evergreen-solar.csv')